## Skill installation

In [ ]:
from selenium import webdriver
import time

from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import utilities
import os

### Helper functions

In [ ]:
FIREFOX_EXE_PATH = 'FIREFOX_EXE_PATH'
GECKO_PATH = 'GECKODRIVER_PATH'

# You can provide link to any skill here
SIGNIN_PAGE = 'LINK_TO_A_SKILL'


DATA_DIR = 'DATA_DIR'

CREDENTIALS_ADDR = os.path.join(DATA_DIR,'CREDENTIALS.json')
SKILLS_ADDR = os.path.join(DATA_DIR,'subgrouped_skills.json')

EXTENSION_PATH = os.path.join(DATA_DIR, 'browser-extension.zip')

#### Update persona name

In [ ]:
PERSONA = 'PERSONA_NAME'

In [ ]:
def get_webdriver(profile_path, gecko_path, exe_path, ext_loc):
    options = FirefoxOptions()
    options.add_argument('-profile')
    options.add_argument(profile_path)
    
    firefox_capabilities = DesiredCapabilities.FIREFOX
    firefox_capabilities['marionette'] = True

    
    firefox_profile = webdriver.FirefoxProfile()
    firefox_bin = FirefoxBinary(firefox_path=exe_path)
    driver = webdriver.Firefox(firefox_binary= firefox_bin,
                               capabilities=firefox_capabilities, 
                               options=options, 
                               executable_path=gecko_path, 
                               service_args=["--marionette-port", "2828"])

    driver.install_addon(ext_loc, temporary=True)
    
    return driver


def quit_driver(driver):
    try:
        driver.quit()
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        
        
def signin(driver, singin_page, email, password):
    try:
        driver.get(singin_page)
        time.sleep(3)

        signin_button = driver.find_element_by_id('a-autoid-0-announce')
        signin_button.click()
        time.sleep(3)
        
        try:
            email_input = driver.find_element_by_id('ap_email')

        except Exception:
            ## Assume that the user is already signed in 
            print('Already signed in. Returning.')
            return True
            
        email_input.send_keys(email)
        time.sleep(3)
        
        continue_button = driver.find_element_by_id('continue')
        continue_button.click()
        time.sleep(3)
        
        password_input = driver.find_element_by_id('ap_password')
        password_input.send_keys(password)
        time.sleep(3)
        
        signin_button = driver.find_element_by_id('signInSubmit')
        signin_button.click()
        time.sleep(3)
        
    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False
    
    return True


def install_skill(driver, url, perm = False):
    try:
        driver.get(url)
        time.sleep(3)
        
        skill_enable_button = driver.find_element_by_id("a2s-skill-enable-button")
        skill_enable_button.click()
        time.sleep(3)
        
        if perm:
            try: 
                permission_button = driver.find_element_by_xpath("//input[@type='submit'][@class='a-button-input'][not(@aria-labelledby)]")
                permission_button.click()
                time.sleep(3)
       
            except Exception:
                print('Something went wrong when enabling permissions')
                return False, 'perm'
            

    except BaseException as ex:
        print('Something went wrong: ', str(ex))
        return False, 'sim'
    
    return True, 'sim'

In [ ]:
def train_persona(profile_path, email, password, skills_json, 
                  no_skills_to_install=50, signin_page=SIGNIN_PAGE,  
                  gecko_path=GECKO_PATH, exe_path=FIREFOX_EXE_PATH, ext_loc=EXTENSION_PATH):
    
    driver = get_webdriver(profile_path, gecko_path, exe_path, ext_loc)
    print('WebDriver registered')
    
    signin_status = signin(driver, signin_page, email, password)

    total_installed = 0
    if signin_status:
        print('Signed in.')
        for skill in skills_json:
            
            skill_perm = all_skills[skill]['Skill_permission']
            skill_url = all_skills[skill]['Skill_link']
            
            if len(skill_perm) > 0:
                skill_perm = True
            else:
                skill_perm = False
                
            
            install_status = install_skill(driver, skill_url, skill_perm)
            
            # Log successfully and partially installed skills
            if install_status[0]:
                total_installed += 1
            
            if total_installed >= no_skills_to_install:
                break
    else:
        print('Could not sign in. Stopping the process')
        quit_driver(driver)
        
    
    quit_driver(driver)
    print(total_installed, len(skills_json))
    print('Installed %f skills.', (total_installed / no_skills_to_install) * 100.0)

### Read email, password, and profile for the persona

In [ ]:
credential = utilities.read_json(CREDENTIALS_ADDR)

email = credential[PERSONA]['email']
password = credential[PERSONA]['password']
profile_addr = credential[PERSONA]['profile']

### Read top skills for the selected persona

In [ ]:
all_skills = utilities.read_json(SKILLS_ADDR)
all_skills = all_skills[PERSONA]

In [ ]:
train_persona(profile_addr, email, password, all_skills)